## Date MCP Server 測試

測試我們建立的 date_server.py MCP Server

In [ ]:
from dotenv import load_dotenv
from agents import Agent, Runner, trace
from agents.mcp import MCPServerStdio
from IPython.display import display, Markdown

load_dotenv(override=True)

### 1. 列出 MCP Server 的 Tools

In [ ]:
params = {"command": "uv", "args": ["run", "date_server.py"]}

async with MCPServerStdio(params=params, client_session_timeout_seconds=30) as server:
    tools = await server.list_tools()
    print("Available tools:")
    for tool in tools:
        print(f"  - {tool.name}: {tool.description}")

### 2. 使用 Agent 呼叫 Date Tool

In [ ]:
instructions = "You can tell the user the current date and time using the available tools."
request = "What is today's date?"
model = "gpt-4.1-mini"

async with MCPServerStdio(params=params, client_session_timeout_seconds=30) as mcp_server:
    agent = Agent(name="date_agent", instructions=instructions, model=model, mcp_servers=[mcp_server])
    with trace("date_agent"):
        result = await Runner.run(agent, request)
    display(Markdown(result.final_output))

### 3. 測試日期時間 Tool

In [ ]:
request = "What is the current date and time?"

async with MCPServerStdio(params=params, client_session_timeout_seconds=30) as mcp_server:
    agent = Agent(name="date_agent", instructions=instructions, model=model, mcp_servers=[mcp_server])
    with trace("date_agent"):
        result = await Runner.run(agent, request)
    display(Markdown(result.final_output))

### 4. 建立 MCP Client 讀取 Resource

In [ ]:
import mcp
from mcp.client.stdio import stdio_client
from mcp import StdioServerParameters

client_params = StdioServerParameters(command="uv", args=["run", "date_server.py"], env=None)

async def read_date_resource():
    async with stdio_client(client_params) as streams:
        async with mcp.ClientSession(*streams) as session:
            await session.initialize()
            result = await session.read_resource("date://today")
            return result.contents[0].text

today = await read_date_resource()
print(today)

### 5. 直接測試 Python 函式（不透過 MCP）

In [ ]:
from datetime import datetime

# 這是 date_server.py 裡面的邏輯
print(f"Date: {datetime.now().strftime('%Y-%m-%d')}")
print(f"DateTime: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")